# VvotingRegression (보팅 회귀)
- 여러 회귀모형 알고리즘을 결합하고 평균 예측값을 반환한다.


## 패키지 로딩

In [1]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

import numpy as np
import pandas as pd


## 데이터 로딩, 스케일링, 분할

In [4]:
boston = pd.read_csv('dataset/HousingData.csv')
boston = boston.fillna(method='ffill')


In [6]:
X = boston.drop('MEDV',axis =1)
y = boston['MEDV']
boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,2.94,36.2


In [7]:
scaled_X = StandardScaler().fit_transform(X)

In [8]:
X_train,X_test,y_train,y_test = train_test_split(scaled_X,y,test_size=0.3,random_state=10)

## 모델 생성

In [9]:
lasso = Lasso(alpha=0.03)
ridge = Ridge(alpha=1)
linear = LinearRegression()

vc_r = VotingRegressor(estimators=[('LASSO',lasso),('RIDGE',ridge),('LINEAR',linear)])
vc_r.fit(X_train,y_train)

VotingRegressor(estimators=[('LASSO', Lasso(alpha=0.03)),
                            ('RIDGE', Ridge(alpha=1)),
                            ('LINEAR', LinearRegression())])

## 예측 및 평가

In [10]:
y_hat = vc_r.predict(X_test)
r_square = vc_r.score(X_test, y_test)
print(f'결정계숫:{r_square:.3f}')
rmse = np.sqrt(metrics.mean_squared_error(y_test,y_hat))
print(f'RMSE:{rmse:.3f}')

결정계숫:0.699
RMSE:5.425


# VotingClassifier (보팅 분류)


## 패키지 로딩

In [11]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


## 데이터 로딩, 정규화, 분할

In [12]:
X, y = load_breast_cancer(return_X_y=True)

scaled_X = StandardScaler().fit_transform(X)


In [13]:
X_train,X_test,y_train,y_test = train_test_split(scaled_X,y,train_size=0.7,random_state=0,stratify=y)

## 모델 생성 및 학습

In [14]:
logistic = LogisticRegression()
knn = KNeighborsClassifier()

# voting: 하드 보팅은 'hard', 소프트 보팅은 'soft'
vo_c = VotingClassifier(estimators=[('LOGISTIC',logistic),("KNN",knn)],voting='soft')
vo_c.fit(X_train,y_train)

VotingClassifier(estimators=[('LOGISTIC', LogisticRegression()),
                             ('KNN', KNeighborsClassifier())],
                 voting='soft')

## 예측 및 평가

In [15]:
y_hat = vo_c.predict(X_test)
print(f'정확도:{metrics.accuracy_score(y_test,y_hat):.3f}')
auc = metrics.roc_auc_score(y_test,vo_c.predict_proba(X_test)[:,1])
print(f'auc는 :{auc:.3f}')

정확도:0.947
auc는 :0.993


# GradientBoostingClassifier(부스팅 분류)
- 랜덤포레스트와 같이 의사결정 나무 모델을 부스팅 방법으로 활용하는 모델
- 이전 예측기가 만든 잔여 오차(residual error)에 ㅐ로운 예측기로 학습 시킴

## 패키지 로딩

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

## 데이터 로딩, 정규화 ,분할

In [16]:
X, y = load_breast_cancer(return_X_y=True)

scaled_X = StandardScaler().fit_transform(X)
X_train,X_test,y_train,y_test = train_test_split(scaled_X,y,train_size=0.7,random_state=0,stratify=y)


## 모델 생성 및 학습

In [20]:
gb_c = GradientBoostingClassifier(random_state=0)
gb_c.fit(X_train,y_train)

GradientBoostingClassifier(random_state=0)

In [23]:
y_hat = gb_c.predict(X_test)
print(f'정확도:{metrics.accuracy_score(y_test,y_hat):.3f}')
auc = metrics.roc_auc_score(y_test,gb_c.predict_proba(X_test)[:,1])
print(f'auc는 :{auc:.3f}')

정확도:0.942
auc는 :0.987


# GradientBoostingRegression (부스팅 회귀)
